In [88]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score
from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn.ensemble import ExtraTreesClassifier  #it split tree randomly with any combination
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import StackingClassifier,StackingRegressor
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from xgboost import XGBClassifier
from sklearn.preprocessing import OneHotEncoder

import os

In [89]:
df=pd.read_csv("train.csv")

In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [91]:
df.isna().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

ct = ColumnTransformer(
    [("norm1", Normalizer(norm='l1'), [0, 1]),
     ("norm2", Normalizer(norm='l1'), slice(2, 4))])

In [92]:
x=df.drop("Loan_Status",axis=1)
y=df["Loan_Status"]

In [93]:
imp_cat=SimpleImputer(strategy="most_frequent")
imp_med=SimpleImputer(strategy="median")

In [94]:
ct=make_column_transformer((imp_cat,make_column_selector(dtype_include=object)),(imp_med,make_column_selector(dtype_exclude=object)),verbose_feature_names_out=False)

In [95]:
ct=ct.set_output(transform="pandas")

In [96]:
x_imp=ct.fit_transform(x)
print(x_imp.isnull().sum().sum())

0


In [97]:

#x_imp = pd.get_dummies(x_imp, drop_first=True)

ohc=OneHotEncoder(sparse_output=False)
ct_enc=make_column_transformer((ohc,make_column_selector(dtype_include=object)),("passthrough",make_column_selector(dtype_exclude=object)),verbose_feature_names_out=False)


In [113]:
ct_enc=ct_enc.set_output(transform="pandas")
x_imp=ct_enc.fit_transform(x_imp)

In [114]:
dtc=DecisionTreeClassifier(random_state=24)
lr=LogisticRegression()
rf=RandomForestClassifier(random_state=24)
xgbm=XGBClassifier(random_state=24)


In [115]:
stack=StackingClassifier([("LR",lr),("DTC",dtc),("RF",rf)],final_estimator=xgbm,passthrough=True)

In [121]:
kfold=StratifiedKFold(n_splits=5,shuffle=True,random_state=24)
params={"DTC__max_depth":[None,3]
        ,"RF__max_features":[3,5,7],
        'final_estimator__n_estimators':[50,100],
        "final_estimator__learning_rate":[0.01,0.1,0.3,0.5],
        "final_estimator__max_depth":[1,3,5] }

In [122]:
gcv=GridSearchCV(stack,param_grid=params,cv=kfold,verbose=3)

In [123]:

gcv.fit(x_imp,y)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


C:\Users\dbda\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\dbda\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

[CV 1/5] END DTC__max_depth=None, RF__max_features=3, final_estimator__learning_rate=0.01, final_estimator__max_depth=1, final_estimator__n_estimators=50;, score=0.805 total time=   1.6s


C:\Users\dbda\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\dbda\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

[CV 2/5] END DTC__max_depth=None, RF__max_features=3, final_estimator__learning_rate=0.01, final_estimator__max_depth=1, final_estimator__n_estimators=50;, score=0.797 total time=   1.7s


...

[CV 4/5] END DTC__max_depth=3, RF__max_features=7, final_estimator__learning_rate=0.5, final_estimator__max_depth=5, final_estimator__n_estimators=100;, score=0.837 total time=   1.8s


C:\Users\dbda\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\dbda\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

[CV 5/5] END DTC__max_depth=3, RF__max_features=7, final_estimator__learning_rate=0.5, final_estimator__max_depth=5, final_estimator__n_estimators=100;, score=0.795 total time=   1.8s


C:\Users\dbda\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\dbda\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=StackingClassifier(estimators=[('LR',
                                                       LogisticRegression()),
                                                      ('DTC',
                                                       DecisionTreeClassifier(random_state=24)),
                                                      ('RF',
                                                       RandomForestClassifier(random_state=24))],
                                          final_estimator=XGBClassifier(base_score=None,
                                                                        booster=None,
                                                                        callbacks=None,
                                                                        colsample_bylevel=None,
                                                                        colsample_bynod...
                                                                        missing=nan,
                                                                        monotone_constraints=None,
                                                                        multi_strategy=None,
                                                                        n_estimators=None,
                                                                        n_jobs=None,
                                                                        num_parallel_tree=None,
                                                                        random_state=24, ...),
                                          passthrough=True),
             param_grid={'DTC__max_depth': [None, 3],
                         'RF__max_features': [3, 5, 7],
                         'final_estimator__learning_rate': [0.01, 0.1, 0.3,
                                                            0.5],
                         'final_estimator__max_depth': [1, 3, 5],
                         'final_estimator__n_estimators': [50, 100]},
             verbose=3)

In [126]:

print(gcv.best_params_)
print(gcv.best_score_)

{'DTC__max_depth': None, 'RF__max_features': 3, 'final_estimator__learning_rate': 0.01, 'final_estimator__max_depth': 1, 'final_estimator__n_estimators': 50}
0.8094762095161935


In [131]:
best_tree=gcv.best_estimator_

In [165]:
test=pd.read_csv("test.csv",)
test.isna().sum().sum()


84

In [166]:
imp_cat=SimpleImputer(strategy="most_frequent")
imp_med=SimpleImputer(strategy="median")

In [167]:
ct=make_column_transformer((imp_cat,make_column_selector(dtype_include=object)),(imp_med,make_column_selector(dtype_exclude=object)),verbose_feature_names_out=False)

In [168]:
ct=ct.set_output(transform="pandas")

In [169]:
test=ct.fit_transform(test)
print(test.isnull().sum().sum())

0


In [ ]:
test.col

In [170]:
ohc=OneHotEncoder(sparse_output=False)
ct_enc=make_column_transformer((ohc,make_column_selector(dtype_include=object)),("passthrough",make_column_selector(dtype_exclude=object)),verbose_feature_names_out=False)


In [171]:
ct_enc=ct_enc.set_output(transform="pandas")
test_imp=ct_enc.fit_transform(test)



In [176]:
test_imp.columns=x_imp.columns

ValueError: Length mismatch: Expected axis has 387 elements, new values have 634 elements

In [172]:
prediction=best_tree.predict(test_imp)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- Loan_ID_LP001015
- Loan_ID_LP001022
- Loan_ID_LP001031
- Loan_ID_LP001035
- Loan_ID_LP001051
- ...
Feature names seen at fit time, yet now missing:
- Loan_ID_LP001002
- Loan_ID_LP001003
- Loan_ID_LP001005
- Loan_ID_LP001006
- Loan_ID_LP001008
- ...


In [173]:
# inferencing 
submit=pd.read_csv("sample_submission_49d68Cx.csv")

In [174]:
submit["Loan_Status"]=

SyntaxError: invalid syntax (1209097062.py, line 1)

#by using 1)decision tree classifier 
2)random forest classifer
3) Logistic Tegression
4) xgBoost  use these algorithm seperately and do the submission